In [1]:
import torch
import csv
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from tqdm import tqdm

import nltk
from nltk.tokenize import sent_tokenize

In [2]:
def get_encode_data(tokenizer, sentences, max_length=256):
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=max_length)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

    return (input_ids, attention_masks)

In [3]:
def get_model(model_name, device, tokenizer=None) :

    print(model_name)

    if tokenizer is None :
      tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


    # parallelization
    if torch.cuda.device_count() > 1:
        print(f'Using {torch.cuda.device_count()} GPUs.')

        model = torch.nn.DataParallel(model)

    return model.to(device), tokenizer

In [4]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [5]:
def seperate_sentence(sentence, chunk) :
    ss = sent_tokenize(sentence)
    
    sentences = []
    merged = ""

    for s in ss :
        if len(merged) + len(s) <= chunk :
            merged = merged + s
        else :
            sentences.append(merged)
            merged = ""
        
    if len(merged) > 256 :
        sentences.append(merged)
    elif len(merged) > 0 :
        sentences[-1] = sentences[-1] + merged
        
    return sentences

In [16]:
def summarize_long_sentence(sentence, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=False) :
    
    ss = seperate_sentence(sentence, chunk)
    summaries = []
    
    for s in ss :
        if verify == True :
            print(s)
        
        summary = summarize(s, tokenizer, model.module, device, max_tokens, min_tokens, num_beams)
        summaries.append(summary)

    return "".join(summaries)

In [17]:
def predict_ad(text, model, tokenizer, s_model, s_tokenizer, device, max_length=256, max_tokens=200, min_tokens=100, num_beams=2) :
    #print(text)
    
    summary = summarize_long_sentence(text, 1024, s_tokenizer, s_model, device, max_tokens, min_tokens, num_beams)
    
    encode_data = get_encode_data(tokenizer, [summary], max_length=max_length)
    output = model(encode_data[0], encode_data[1])
    _, predicted = torch.max(output.logits, dim=1)
    predicted = predicted.item()
    
    if predicted : 
        return True
    else :
        return False

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [8]:
#set up summry model
max_tokens = 200
min_tokens = 100
num_beams = 2

s_model, s_tokenizer = get_model('psyche/KoT5-summarization', device)

psyche/KoT5-summarization
Using 2 GPUs.


In [9]:
#set up ad check model
model_name = 'klue/roberta-base'
b_model=128
length=256

pretrained_file = '/home/osung/work/lm_classification/klue_roberta-base_ad_total.tsv_ad_b128_e10_ml256_20240814_135702.pth'
num_labels = 2

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False,)
pretrained_model_config = AutoConfig.from_pretrained(model_name)
pretrained_model_config.num_labels = num_labels

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=pretrained_model_config,
)

model = torch.nn.DataParallel(model)
model = model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
loaded_state_dict = torch.load(pretrained_file)
model.load_state_dict(loaded_state_dict)

<All keys matched successfully>

In [12]:
text = '부동산 재테크 열기, 주목할 만한 곳은? 국내 실물경기 회복에 따라 올해 하반기 이후부터는 부동산시장이 수도권을 중심으로 본격적인 상승세를 나타낼 것으로 전망되고 있다. 다만 보금자리주택 공급확대와 준주택 도입, 위례신도시 분양, 지방선거 등 주요 현안들이 가시화 될 전망이어서 부동산 각 분야별로 상승 분위기는 차이가 있을 것으로 보인다. 그렇다면 올해 재테크 열기를 이어갈 주목할만한 부동산 투자처는 어디일까?- 2월 11일 세제 마감, 주목 받는 미분양 아파트는?교통호재와 개발호재가 있는 지역의 수도권 아파트 시장은 올해에도 꾸준한 인기를 끌 것으로 예상된다. 그 중에서도 2월 11일 정부의 세제 마감 시한을 앞둔 역세권 주변의 미분양 아파트들은 현재 마지막 수요가 몰리고 있다. 부동산전문가들은 할인혜택, 경품을 내건 미분양아파트에 혹하기보다는 향후 가치상승이 기대되는 곳에 분양 받을 것을 권하고 있다. ◇GS건설 서초 아트 자이GS건설이 서초동에 지은 서초아트자이를 특별 분양 하고 있다. 이 주상복합은 지하 4층, 지상 25층의 4개동으로 △181.10㎡(54평형)56가구 △207.49㎡(62평형)104가구 △336.71㎡(101평형) 4가구, 오피스텔 26가구 등 총 190가구 규모다. 서초 아트자이는 전용률이 80%이고 양면 개방형 신평면 구조를 채택해 통풍채광이 뛰어나고 전 가구가 방향이 다양하게 배치되었다. 3호선 남부터미널역과 2,3호선 교대역이 가까워 교통여건이 뛰어나며 서초 남부터미널 개발과, 롯데타운 개발예정에 따른 파급효과도 기대 할 수 있다. 또 서초아트자이는 주거지역, 중심업무지구, 문화예술공간이 공존하는 강남의 복합상권으로 시너지효과를 극대화할 수 있는 입지가 장점이다. 내부 주거 환경도 뛰어나 지상 22층 높이 스카이 브릿지에 170평 규모의 휘트니스센터와 게스트룸, 클럽하우스 등 자이안센터가 있으며 강남 시내와 관악산이 한눈에 보이는 자이만의 독특한 조망권을 느낄 수 있다. 6월 31일까지 취.등록세 감면된다. (분양문의:02-585-2698)◇한강신도시 ‘우미린’ 김포한강신도시에 공급되는 우미건설 우미린은 건설사 신용위험평가 A등급의 건설사로서 지하 2층 지상 26층 , 1058세대 131.8㎡ (39평),156.03㎡(47평)평형을 1,2,3순위 청약을 마치고 일반분양에 들어간다. 우미린은 올림픽대로와 연결된 김포고속화도로 와 4번 국도 확장으로 자동차를 이용해 여의도까지 약 30분대 진입이 가능하며 단지에서 차량 약 10분 거리에 제2외곽순환도로가 연결되어 일산,청라, 송도까지 접근성도 우수하다. 지하철은 5,9호선과 연결되는 경전철역과 중심상업지구가 도보 약 5분 거리에 위치하며 단지 인근 에는 3만 3천평 규모의 호수공원과 다양한 문화 및 상업시설이 근접해있다. 총 14개 동으로 구성된 우미린의 계약시 조건은 계약금 5%, 중도금 전액 무이자(단, 일부 주택형별로 상이함)융자이며,선착순으로 수의계약으로 청약 통장 없이 계약이 가능하다. 오는 2월11일까지 계약하는 경우 양도세가 5년간 100% 면제되며 전매제한 기간도 전용 85㎡ 초과여서 올 7월부터 전매가 가능하다. 입주는 2011년 10월. (분양문의: 070-7609-4400)◇파주교하 한양수자인 `파주 한양수자인`은 지하 3층~지상 26층 총 13개 동 규모로 85㎡ 72가구, 108㎡ 358가구, 149㎡ 350가구로 특별 분양 중이다. 지난 17일 오픈한 모델하우스에는 첫날부터 2만5천여명이 방문하며 북새통을 이뤘다. 양도소득세는 분양 후 5년간 100%가 면제되며 전매제한도 1~3년에 불과하다. 경의선 운정역과 가깝고 주변에 초등•중학교가 위치해 있다. 특히 한양수자인 아파트가 인접해 있는 운정역 인근은 대규모 복합쇼핑몰과 백화점, 할인점 등 대형 상업시설을 비롯해 비즈니스 호텔과 각종 전시장, 문화시설 등 다양한 인프라가 들어설 예정이어서 기반시설도 우수하다는 평가를 받고 있다. 교하 한양수자인의 중소형 아파트는 3.3㎡당 분양가가 890만~990만원대로 1100~1300만원대에 이르는 주변아파트 보다 시세는 저렴한 편이다. 파주시 야당리 부동산 관계자는 "한양수자인의 경우에는 미분양 물량으로 인해 5년간 양도세가 면제되고 파주신도시는 이번 경의선 개통과 제2외곽순환도로, 김포~관산간 등 교통인프라가 확충될 예정이어서 서울 강북권과 도심으로의 접근성이 크게 개선돼 실수요자들의 각광을 받을 것으로 예상된다"고 덧붙였다. 오는 2012년 1월 입주 예정이다. (분양문의: 031-919-8125) -전망 밝은 소형 오피스텔 활기 오피스텔은 올해 대폭적인 규제 완화로 전망이 가장 밝다. 현재 오피스텔에 대한 정부의 전폭적인 정책지원과 희소성이 부각되면서 역세권 등 위치가 좋고 임대수요가 많은 지역의 오피스텔은 활기를 띄고 있다. ◇부천시 중동 미니오피스텔 로하스 레지던스부천시 중동에 분양중인 미니오피스텔 로하스 레지던스의 경우 모델 하우스 오픈 한 달 만에 이미 50%이상 계약을 완료했다. 주위에 많은 학원가가 밀집되어 있고 부천대학교, 부천시청, 홈플러스, GS백화점, 현대백화점 등이 가까이 있으며 주변으로 호텔과 높은 고층의 업무시설이 들어설 예정이어서 입지와 수요 면에서는 유리하다. 특히, 풀 옵션을 갖춘 로하스 레지던스는 내년 말 개통될 7호선 연장선(부천 중부경찰서역)을 이용하면 서울 강남까지 40분이면 출 퇴근 가능해 교통여건이 뛰어나다. 로하스 레지던스 관계자는 “ 내년 말 개통될 7호선 호재가 겹친다면 지가상승이 예상될 뿐만 아니라 3,000만원대 투자로 매달 40만~50만원의 임대 수익이 예상된다”고 밝혔다. 3월말 입주예정 (분양문의 : 032- 321-1761)'

In [18]:
predict_ad(text, model, tokenizer, s_model, s_tokenizer, device)

True